In [1]:
from PIL import Image
import numpy as np
import cv2

In [2]:
def clearing_background(path_to_image: str) -> (Image.Image, np.array):
    """
    Функция для очистки фона у фундус снимка
    :param path_to_image: Путь до изображения  
    :return: (Изображение с прозрачным фоном, маска обрезки)
    """
    # Первичная обработка снимка, добавляем альфа канал
    eye = cv2.imread(path_to_image, cv2.IMREAD_UNCHANGED)
    eye = cv2.cvtColor(eye, cv2.COLOR_BGR2BGRA)
    # Немного затемняем и получаем чб
    contrast_eye = cv2.convertScaleAbs(eye, alpha=0.5, beta=100)
    gray_contranst_eye = cv2.cvtColor(contrast_eye, cv2.COLOR_BGR2GRAY)
    
    # Выделяем объекты с помощью THRESH_TRIANGLE
    triangle_eye = cv2.threshold(gray_contranst_eye, 0, 255, cv2.THRESH_TRIANGLE)[1]
    # Используем морф, чтобы почистить от шумов и сделать контуры более гладкими
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    triangle_morph_eye = cv2.morphologyEx(triangle_eye, cv2.MORPH_OPEN, kernel_ellipse, iterations=5)
    
    # Выделяем контуры на снимке, и выбираем самый большой (по идее это глаз)
    contours_eye = cv2.findContours(triangle_morph_eye, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    max_contour_eye = max(contours_eye, key=cv2.contourArea)
    # На основе конура создаем маску 
    mask_eye = np.zeros_like(eye)
    cv2.fillConvexPoly(mask_eye, max_contour_eye, (255, 255, 255, 255))
    
    clear_eye = cv2.cvtColor(cv2.bitwise_and(eye, mask_eye), cv2.COLOR_BGRA2RGBA)
    
    return Image.fromarray(clear_eye), mask_eye